## Entendendo como extrair informações sobre repositórios no github

[Search](https://developer.github.com/v3/search/)
[Searching for repositories](https://help.github.com/en/articles/searching-for-repositories#search-by-repository-name-description-or-contents-of-the-readme-file)


Lista de qualifiers:  https://help.github.com/en/articles/searching-code
Documentação para search: https://developer.github.com/v3/search/

In [121]:
import requests
import pandas as pd

Montando url de busca para os repositórios com maior quantidade de estrelas.
Usando como consulta stars:>1 para indicar que são todos os repositorios acima de 1 estrela, o que não tem impacto negativo para a busca pois existem certa de 3.305.358 repositórios e só vamos analisar 2.500.

In [122]:
q = 'q=stars:>1'
sort = '&sort=stars&order=desc'
url_base = 'https://api.github.com/search/repositories?'
url_final = url_base+q+sort
url_final

'https://api.github.com/search/repositories?q=stars:>1&sort=stars&order=desc'

In [123]:
colunas=['owner_type', 
           'owner_url',
           'owner_html_url',
           'html_url',
           'url',
           'fork',
           'created_at',
           'updated_at',
           'size',
           'stargazers_count',
           'language',
           'has_issues',
           'has_wiki',
           'forks_count',
           'forks',
           'open_issues',
           'watchers']

In [124]:
resultados = pd.DataFrame(columns=['owner_type', 
                                   'owner_url'
                                   'owner_html_url',
                                   'html_url',
                                   'url',
                                   'fork',
                                   'created_at',
                                   'updated_at',
                                   'size',
                                   'stargazers_count',
                                   'language',
                                   'has_issues',
                                   'has_wiki',
                                   'forks_count',
                                   'forks',
                                   'open_issues',
                                   'watchers',
                                   'commits',
                                   'contributors',
                                   'readme',])
resultados.head()

,owner_type,owner_urlowner_html_url,html_url,url,fork,created_at,updated_at,size,stargazers_count,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,commits,contributors,readme


In [125]:
def add_resultado(item):
    df = pd.DataFrame([[
                        item.get('owner').get('type', None),
                        item.get('owner').get('url', None),
                        item.get('owner').get('html_url', None),
                        item.get('html_url', None),
                        item.get('url', None),
                        item.get('fork', None),
                        item.get('created_at', None),
                        item.get('updated_at', None),
                        item.get('size', None),
                        item.get('stargazers_count', None),
                        item.get('language', None),
                        item.get('has_issues', None),
                        item.get('has_wiki', None),
                        item.get('forks_count', None),
                        item.get('forks', None),
                        item.get('open_issues', None),
                        item.get('watchers', None)
                    ]], columns=colunas)

    return df    

In [170]:
def extrair_dados(data, resultados):
    for item in data.get('items', None):
        resultados = pd.concat([resultados, add_resultado(item)], ignore_index=True, sort=False)
        
    return resultados

In [171]:
def get_total_paginas(data):
    
    itens_por_pagina = len(data.get('items'))
    total_paginas = data.get('total_count') // itens_por_pagina # opreração de div em python
    total_paginas

    print('Total de registros:{0} , Registros por página:{1}, Total de Páginas:{2}'.format(
        data.get('total_count'), itens_por_pagina, total_paginas))
    
    return total_paginas        

In [190]:
def percorrendo_paginas(url):
    
    resultados = pd.DataFrame(columns=['owner_type', 
                                   'owner_url'
                                   'owner_html_url',
                                   'html_url',
                                   'url',
                                   'fork',
                                   'created_at',
                                   'updated_at',
                                   'size',
                                   'stargazers_count',
                                   'language',
                                   'has_issues',
                                   'has_wiki',
                                   'forks_count',
                                   'forks',
                                   'open_issues',
                                   'watchers',
                                   'commits',
                                   'contributors',
                                   'readme',])
    
    results = requests.get(url)    
    data = dict(results.json())
    extrair_dados(data, resultados)

    # iteracoes = get_total_paginas(data)
    iteracoes = 84 # Para pegar em torno de 2500 registros, semelhante ao artigo original
    
    for iteracao in range(1, 20):
        print('Extraindo página:{0}'.format(iteracao))
        
        header = dict(results.links)
        next_url = header.get('next').get('url')
        
        results = requests.get(next_url)
        print('Status:{0}'.format(results))
        
        data = dict(results.json())
        print('Data keys:{0}'.format(data.keys()))
            
        resultados = extrair_dados(data, resultados)
        
    return resultados

In [191]:
resultados = percorrendo_paginas(url_final)

Extraindo página:1
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
Extraindo página:2
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
Extraindo página:3
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
Extraindo página:4
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
Extraindo página:5
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
Extraindo página:6
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
Extraindo página:7
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
Extraindo página:8
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
Extraindo página:9
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
E

TypeError: 'NoneType' object is not iterable

In [192]:
resultados.describe()

,owner_type,owner_urlowner_html_url,html_url,url,fork,created_at,updated_at,size,stargazers_count,language,...,has_wiki,forks_count,forks,open_issues,watchers,commits,contributors,readme,owner_url,owner_html_url
count,270,0.0,270,270,270,270,270,270,270,232,...,270,270,270,270,270,0.0,0.0,0.0,270,270
unique,2,0.0,270,270,1,270,265,269,267,27,...,2,269,269,223,267,0.0,0.0,0.0,252,252
top,Organization,NaN,https://github.com/gitlabhq/gitlabhq,https://api.github.com/repos/godotengine/godot,False,2014-10-16T18:29:12Z,2019-06-25T14:26:02Z,440,21187,JavaScript,...,True,3445,3445,39,21187,NaN,NaN,NaN,https://api.github.com/users/apache,https://github.com/apache
freq,170,NaN,1,1,270,1,2,2,2,86,...,163,2,2,4,2,NaN,NaN,NaN,4,4


In [184]:
resultados.to_csv('../dados/repositorios.csv', index=False)